In [1]:
import gzip
import shutil
import requests
import json
from bs4 import BeautifulSoup as bs
import pandas as pd
import os


In [2]:
general_url = 'http://jmcauley.ucsd.edu/data/amazon/'
data_url = 'http://deepyeti.ucsd.edu/jianmo/amazon/index.html'

# Scrape for the web page

In [3]:

results = requests.get(data_url)
src = results.content
src

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"\n"http://www.w3.org/TR/html4/loose.dtd">\n<html>\n  <head>\n    <title>Amazon review data</title>\n    <meta name="author" content="Jianmo Ni">\n    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n    <script type="text/javascript" language="javascript"  src="http://code.jquery.com/jquery-1.4.2.min.js"></script>\n    <link rel="stylesheet" type="text/css" href="https://cseweb.ucsd.edu/~jmcauley/style.css">\n  </head>\n<body>\n\n<div id="left_column">\n</div>\n\n<div id="main">\n<!-- <div id="header"> -->\n  <h1>Amazon review data (2018)</h1>\n\n  <p><b><a href="https://nijianmo.github.io">Jianmo Ni</a></b>, UCSD</p>\n\n<h2>Citation</h2>\n<p>Please cite the following paper if you use the data in any way:</p>\n<p>\n  <b>Justifying recommendations using distantly-labeled reviews and fined-grained aspects</b><br>\n  Jianmo Ni, Jiacheng Li, Julian McAuley<br>\n  <em>Empirical Methods in Natural Language Pr

In [4]:
soup = bs(src, 'lxml')
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<title>Amazon review data</title>
<meta content="Jianmo Ni" name="author"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script language="javascript" src="http://code.jquery.com/jquery-1.4.2.min.js" type="text/javascript"></script>
<link href="https://cseweb.ucsd.edu/~jmcauley/style.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<div id="left_column">
</div>
<div id="main">
<!-- <div id="header"> -->
<h1>Amazon review data (2018)</h1>
<p><b><a href="https://nijianmo.github.io">Jianmo Ni</a></b>, UCSD</p>
<h2>Citation</h2>
<p>Please cite the following paper if you use the data in any way:</p>
<p>
<b>Justifying recommendations using distantly-labeled reviews and fined-grained aspects</b><br/>
  Jianmo Ni, Jiacheng Li, Julian McAuley<br/>
<em>Empirical Methods in Natural Language Processing (EMNLP)</em>, 2019<br/>
<a href="http://cseweb.ucsd

In [5]:
table_soup = soup.find('table',{'class':'code-table'})
table_soup

<table class="code-table" style="font-size: 0.7em; width:90%">
<tr>
<td>AMAZON FASHION</td>
<td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/AMAZON_FASHION.json.gz">reviews</a> (883,636 reviews)</td>
<td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_AMAZON_FASHION.json.gz">metadata</a> (186,637 products)</td>
</tr>
<tr>
<td>All Beauty</td>
<td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/All_Beauty.json.gz">reviews</a> (371,345 reviews)</td>
<td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz">metadata</a> (32,992 products)</td>
</tr>
<tr>
<td>Appliances</td>
<td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Appliances.json.gz">reviews</a> (602,777 reviews)</td>
<td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Appliances.json.gz">metadata</a> (30,459 products)</td>
</tr>
<tr>
<td>Arts Crafts and Sewing</td>
<td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/categor

In [6]:
td_soup = table_soup.find_all('td')
td_soup

[<td>AMAZON FASHION</td>,
 <td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/AMAZON_FASHION.json.gz">reviews</a> (883,636 reviews)</td>,
 <td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_AMAZON_FASHION.json.gz">metadata</a> (186,637 products)</td>,
 <td>All Beauty</td>,
 <td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/All_Beauty.json.gz">reviews</a> (371,345 reviews)</td>,
 <td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz">metadata</a> (32,992 products)</td>,
 <td>Appliances</td>,
 <td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Appliances.json.gz">reviews</a> (602,777 reviews)</td>,
 <td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Appliances.json.gz">metadata</a> (30,459 products)</td>,
 <td>Arts Crafts and Sewing</td>,
 <td><a href="http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Arts_Crafts_and_Sewing.json.gz">reviews</a> (2,875,917 reviews)</td>,
 <t

In [7]:
td_soup[1].find('a').text

'reviews'

# Scrape for the downloading list

In [8]:
# temp_td_soup = td_soup[0].find('a').get('href')
list_data_url = []
list_meta_data_url = []

for i in range(len(td_soup)):
    try:
        temp_td_soup = td_soup[i].find('a').get('href')

        # if the url label is meta-data
        if td_soup[i].find('a').text == 'metadata':
            list_meta_data_url.append(temp_td_soup)
            print(f"No{i}. {temp_td_soup}, meta-data")

        # if the url label is data
        elif td_soup[i].find('a').text == 'reviews':
            list_data_url.append(temp_td_soup)
            print(f"No{i}. {temp_td_soup}, reviews")

    except AttributeError:
        print(f'No{i}. No href')


No0. No href
No1. http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/AMAZON_FASHION.json.gz, reviews
No2. http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_AMAZON_FASHION.json.gz, meta-data
No3. No href
No4. http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/All_Beauty.json.gz, reviews
No5. http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz, meta-data
No6. No href
No7. http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Appliances.json.gz, reviews
No8. http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Appliances.json.gz, meta-data
No9. No href
No10. http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Arts_Crafts_and_Sewing.json.gz, reviews
No11. http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Arts_Crafts_and_Sewing.json.gz, meta-data
No12. No href
No13. http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Automotive.json.gz, reviews
No14. http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Automotive.json.gz, meta-data
No15. No href
No16

# get the list of saving dir

In [9]:
list_data_dir = [i.split('/')[-1] for i in list_data_url]
list_meta_data_dir = [i.split('/')[-1] for i in list_meta_data_url]
list_data_dir

['AMAZON_FASHION.json.gz',
 'All_Beauty.json.gz',
 'Appliances.json.gz',
 'Arts_Crafts_and_Sewing.json.gz',
 'Automotive.json.gz',
 'Books.json.gz',
 'CDs_and_Vinyl.json.gz',
 'Cell_Phones_and_Accessories.json.gz',
 'Clothing_Shoes_and_Jewelry.json.gz',
 'Digital_Music.json.gz',
 'Electronics.json.gz',
 'Gift_Cards.json.gz',
 'Grocery_and_Gourmet_Food.json.gz',
 'Home_and_Kitchen.json.gz',
 'Industrial_and_Scientific.json.gz',
 'Kindle_Store.json.gz',
 'Luxury_Beauty.json.gz',
 'Magazine_Subscriptions.json.gz',
 'Movies_and_TV.json.gz',
 'Musical_Instruments.json.gz',
 'Office_Products.json.gz',
 'Patio_Lawn_and_Garden.json.gz',
 'Pet_Supplies.json.gz',
 'Prime_Pantry.json.gz',
 'Software.json.gz',
 'Sports_and_Outdoors.json.gz',
 'Tools_and_Home_Improvement.json.gz',
 'Toys_and_Games.json.gz',
 'Video_Games.json.gz']

# Create the saving dir

In [10]:
if not os.path.exists('./meta'):
    os.mkdir('./meta')
    print('Created dir for meta')
else:
    print('dir for meta exists')

if not os.path.exists('./review'):
    os.mkdir('./review')
    print('Created dir for review')
else:
    print('dir for review exists')




Created dir for meta
Created dir for review


In [11]:
for i in range(len(list_data_url)):
    # download the items
    print(f'Downloading: {list_data_url[i]}')
    temp_downloaded = requests.get(list_data_url[i])
    # get the save dir of gzip file
    gzip_save_dir = f'review/{list_data_dir[i]}'
    # get the save dir for the json file
    json_save_dir = gzip_save_dir[:-3]

    open(gzip_save_dir, 'wb').write(temp_downloaded.content)

    with gzip.open(gzip_save_dir, 'rb') as f_in:
        with open(json_save_dir, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    os.remove(gzip_save_dir)

Downloading: http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/AMAZON_FASHION.json.gz
Downloading: http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/All_Beauty.json.gz
Downloading: http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Appliances.json.gz
Downloading: http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Arts_Crafts_and_Sewing.json.gz
Downloading: http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Automotive.json.gz
Downloading: http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Books.json.gz


KeyboardInterrupt: 

In [ ]:
for i in range(len(list_meta_data_dir)):
    print(f"Downloading: {list_meta_data_url[i]}")
    # download the items
    temp_downloaded = requests.get(list_meta_data_url[i])
    # get the save dir of gzip file
    gzip_save_dir = f'meta/{list_meta_data_dir[i]}'
    # get the save dir for the json file
    json_save_dir = gzip_save_dir[:-3]

    open(gzip_save_dir, 'wb').write(temp_downloaded.content)

    with gzip.open(gzip_save_dir, 'rb') as f_in:
        with open(json_save_dir, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    os.remove(gzip_save_dir)

In [ ]:
json

In [ ]:
temp_df = pd.read_json(json_save_dir)